# IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装

このノートブックは、研究報告論文「IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を部分適用する方式の提案と実装」における提案方式の実装を、既存方式と比較検討した際のプロトコルを記録しておくためのものである。

計測方法について、北九州市立大学の山崎進氏による記事「[mix_tasks_upload_hotswap の Hexライブラリ版を試す](https://qiita.com/zacky1972/items/f0b47eded7c902008871)」がおおいに参考になりました。感謝いたします。

## 更新対象とするIoTアプリケーション

各方式の比較検討に用いるアプリケーションは、本リポジトリの[target_app](./target_app)ディレクトリに収められている。そのうちの、以下に抜粋したコードに含まれる`TargetApp.hello/0`メソッドが返す文字列を変更することで、コードの変更とする。

```
defmodule TargetApp do
  def hello do
    :world
  end
end
```

## 比較検討する方式

上記のIoTアプリケーションコードの変更をIoTデバイスに適用する方式として、以下の3つについて比較検討する。

1. ファームウェアイメージ全体の適用
2. ファームウェアイメージのパッチによる部分適用
3. アプリケーションコードの部分適用

それぞれの詳細については、研究報告論文を参照されたい。

## 比較検討の方法

各方式を比較するための指標として、開発者によるコードの変更をIoTデバイスに適用し、アプリケーションが通信可能となるまでの時間を用いる。

方式を実装したIoTデバイス開発プラットフォームとして、Nervesを用いる。筆者による提案方式3.の実装を加えると、上記の方式をすべて実装しているのがNervesだからである。ハードウェアとしては、Raspberry Piを用いる。

方式1.および2.については、スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下の時間を計測する。

1. ファームウェアイメージの生成
2. ファームウェアイメージのデバイスへのアップロード
3. デバイスの再起動

方式3.については、提案方式を実装した`mix upload.hotswap`コマンドの実行時間を、`time(1)`によって計測する。

## 実験環境

### 開発環境

* MacBook Pro (13-inch, 2018, Four Thunderbolt 3 Ports)
* プロサッサ: 2.7 GHz クアッドコアIntel Core i7
* メモリ: 16 GB 2133 MHz LPDDR3

### IoTデバイス

* Raspberry Pi 3 Model B
* 開発環境との通信: 有線LAN

### Elixir/Nervesのバージョン

* Nerves: 1.7.1
* Nerves Bootstrap: 1.10.1
* Elixir: 1.11.2

このNotebookでは、Nervesの開発環境の整備については解説しない。「[ElixirでIoT#4.1：Nerves開発環境の準備（2020年11月版）](https://qiita.com/takasehideki/items/88dda57758051d45fcf9)」等の記事を参照されたい。

## 実験

### 1. ファームウェアイメージ全体の適用

スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下に要する時間を計測する。

1. `mix firmware`
2. `mix upload`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

3回計測を行い、それらの平均を用いる。

#### 1回目

In [4]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m21.927s
user	0m6.584s
sys	0m2.121s

Time to upload firmware

real	0m14.572s
user	0m5.182s
sys	0m1.261s

Time to reboot

time: 22.692 sec.



#### 2回目

In [1]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m20.573s
user	0m6.462s
sys	0m2.150s

Time to upload firmware

real	0m14.527s
user	0m5.599s
sys	0m1.337s

Time to reboot

time: 22.522 sec.



#### 3回目

In [5]:
!cd target_app && ../measure_firmware_update.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m21.081s
user	0m6.499s
sys	0m2.020s

Time to upload firmware

real	0m14.328s
user	0m5.159s
sys	0m1.214s

Time to reboot

time: 22.880 sec.



#### まとめ

In [11]:
print("1. ファームウェアイメージの生成: {:.2f}秒".format((21.927+20.573+21.081)/3))
print("2. ファームウェアイメージのデバイスへのアップロード: {:.2f}秒".format((14.572+14.527+14.328)/3))
print("3. デバイスの再起動: {:.2f}秒".format((22.692+22.522+22.880)/3))

1. ファームウェアイメージの生成: 21.19秒
2. ファームウェアイメージのデバイスへのアップロード: 14.48秒
3. デバイスの再起動: 22.70秒


### 2. ファームウェアイメージのパッチによる部分適用

スクリプト[measure_firmware_update.sh](./measure_firmware_update.sh)を用いて、以下に要する時間を計測する。

1. `mix firmware.patch`
2. `mix upload --firmware _build/rpi3_dev/nerves/images/patch.fw`
3. Nervesが再起動して`ping(1)`が通るようになるまでの時間

3回計測を行い、それらの平均を用いる。

#### 1回目

In [8]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m27.127s
user	0m6.697s
sys	0m2.227s

Time to upload firmware

real	0m17.873s
user	0m3.897s
sys	0m0.952s

Time to reboot

time: 22.534 sec.



#### 2回目

In [2]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m23.531s
user	0m6.589s
sys	0m2.185s

Time to upload firmware

real	0m18.240s
user	0m3.959s
sys	0m1.007s

Time to reboot

time: 22.743 sec.



#### 3回目

In [13]:
!cd target_app && ../measure_firmware_update.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m22.494s
user	0m6.610s
sys	0m2.180s

Time to upload firmware

real	0m17.922s
user	0m4.169s
sys	0m1.062s

Time to reboot

time: 22.638 sec.



#### まとめ

In [17]:
print("1. ファームウェアイメージの生成: {:.2f}秒".format((27.127+23.531+22.494)/3))
print("2. ファームウェアイメージのデバイスへのアップロード: {:.2f}秒".format((17.873+18.240+17.922)/3))
print("3. デバイスの再起動: {:.2f}秒".format((22.534+22.743+22.638)/3))

1. ファームウェアイメージの生成: 24.38秒
2. ファームウェアイメージのデバイスへのアップロード: 18.01秒
3. デバイスの再起動: 22.64秒


### 3. アプリケーションコードの部分適用

1. `mix upload.hotswap`

この方式ではNervesの再起動は必要ないため、適用の手順は以上のみである。上記のコマンドの実行に要する時間を、`time(1)`によって計測する。

3回計測を行い、それらの平均を用いる。

#### 1回目

In [11]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m3.414s
user	0m4.708s
sys	0m1.322s


#### 2回め

In [15]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m3.197s
user	0m4.668s
sys	0m1.256s


#### 3回目

In [16]:
!cd target_app && time mix upload.hotswap > /dev/null


real	0m2.915s
user	0m4.528s
sys	0m1.200s


#### まとめ

In [18]:
print("1. `mix upload.hotswap`: {:.2f}秒".format((3.414+3.197+2.915)/3))

1. `mix upload.hotswap`: 3.18秒


### 各方式の計測結果のまとめ

|                         | firmware build | firmware upload | reboot | code update | sum   |
|-------------------------|----------------|-----------------|--------|-------------|-------|
| firmware update (full)  |          21.19 |           14.48 |  22.70 |           - | 58.37 |
| firmware update (patch) |          24.38 |           18.01 |  22.64 |           - | 65.04 |
| proposed method         |              - |               - |      - |        3.18 |  3.18 |